In [3]:
import kagglehub
import os
import shutil

target_folder = "/home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data"

os.makedirs(target_folder, exist_ok=True) 

print("Đang tải dataset vào cache của kagglehub...")
try:
    cache_path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")
    print(f"Dataset đã được tải về cache tại: {cache_path}")

    print(f"Đang di chuyển tệp từ cache vào thư mục '{target_folder}'...")

    for item_name in os.listdir(cache_path):
        source_item = os.path.join(cache_path, item_name)
        print(f"Soucre item: {source_item}")
        destination_item = os.path.join(target_folder, item_name)

        if os.path.exists(destination_item):
            if os.path.isdir(destination_item):
                shutil.rmtree(destination_item)
            else:
                os.remove(destination_item)

        shutil.move(source_item, destination_item)


    print(f"Hoàn tất! Dữ liệu đã được chuyển đến: {os.path.abspath(target_folder)}")
    print(f"Các tệp trong thư mục '{target_folder}':")
    for item in os.listdir(target_folder):
        print(f"- {item}")

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")
    print("Vui lòng kiểm tra kết nối mạng và xác thực Kaggle của bạn.")
    print("Hãy chắc chắn bạn đã đặt file kaggle.json hoặc dùng kagglehub.login().")

Đang tải dataset vào cache của kagglehub...
Dataset đã được tải về cache tại: /home/tiamo/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2
Đang di chuyển tệp từ cache vào thư mục '/home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data'...
Hoàn tất! Dữ liệu đã được chuyển đến: /home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data
Các tệp trong thư mục '/home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data':
- twitter_validation.csv
- twitter_training.csv.zip
- twitter_training.csv


In [12]:
import os
import pandas as pd
path = os.path.join(os.getcwd(),'data/twitter_training.csv')
header = ['label','text']
df = pd.read_csv(path)
df = df.iloc[:,2:4]
df.columns = header
df

,label,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [19]:
import kagglehub
import nltk
import re
import os
import pandas as pd

class TwitterDataset:
    def __init__(self,file_path):
        self.file_path = file_path
        self.df = None
        
    def download_data(self):
        data = pd.read_csv(os.path.join(os.getcwd(),self.file_path))
        header = ['label','text']
        self.df = pd.DataFrame(data.iloc[:,2:4])
        self.df.columns = header
        
    def prepocess_data(self):
        pass
    
twitter = TwitterDataset('data/twitter_training.csv')
twitter.download_data()
twitter.df['label'].value_counts()

label
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer


sentences = [
    "machine learning is great",
    "natural language processing and machine learning are fun",
    "gensim is a useful library for text processing"
]

processed_sentences = [simple_preprocess(sentence) for sentence in sentences]

vectorizer = TfidfVectorizer(lowercase=False)

vectorizer.fit(sentences)
vectorizer.vocabulary_.items()
tmp = {word: vectorizer.idf_[i] for word, i in vectorizer.vocabulary_.items()}
tmp.get('machine')



# model = Word2Vec(sentences=processed_sentences, vector_size=100, window=5, min_count=1, workers=4,epochs=20)

vector = model.wv['machine']  
similar_words = model.wv.most_similar('machine') 
print(similar_words)

1.2876820724517808

In [5]:
import numpy as np
import pandas as pd
from dataLoader import TwitterDataset
from processor import TextProcessor
from model.tfidf_model import TfIdfModel
from model.word2vec import Word2VecMLP
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

train_loader = TwitterDataset('data/twitter_training.csv')
train_loader.load_data()
train_loader.prepocess_label()
df_train = train_loader.df

test_loader = TwitterDataset('data/twitter_validation.csv')
test_loader.load_data()
test_loader.prepocess_label()
df_test = test_loader.df


preprocessor = TextProcessor()
df_train,df_test = preprocessor.preproces_dataframe(df_train),preprocessor.preproces_dataframe(df_test)

# TFIDF - Model
# tfidf_model = TfIdfModel()
# tfidf_model.train(df_train['clean text'],df_train['label'])
# tfidf_model.evaluate(df_test['clean text'],df_test['label'])


df_train["tokenized"] = df_train["clean text"].apply(lambda x: word_tokenize(x.lower()))
df_test['tokenized'] = df_test['clean text'].apply(lambda x: word_tokenize(x.lower()))

X_train,y_train = df_train['tokenized'].tolist(),df_train['label'].tolist()
X_test,y_test = df_test['tokenized'].tolist(),df_test['label'].tolist()

# X_train,y_train = np.array(X_train,dtype=np.float32),np.array(y_train,dtype=np.float32)
# X_test,y_test = np.array(X_test,dtype=np.float32),np.array(y_test,dtype=np.float32)


word2vec_mlp = Word2VecMLP(mlp_epochs=100,word2vec_epochs=10)
word2vec_mlp.train_word2vec(X_train)
tfidf_weights = word2vec_mlp.compute_tfidf_weights(X_train)

print(word2vec_mlp.device)


Training Word2Vec for 10 epochs...


/home/tiamo/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


cpu


In [6]:
word2vec_mlp.train(X_train,y_train,tfidf_weights)
word2vec_mlp.evaluate(X_test,y_test,tfidf_weights)

Training MLP


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Loss: 1.3948
Epoch [11/100], Loss: 1.1342
Epoch [21/100], Loss: 1.0858
Epoch [31/100], Loss: 1.0549
Epoch [41/100], Loss: 1.0274
Epoch [51/100], Loss: 0.9995
Epoch [61/100], Loss: 0.9717
Epoch [71/100], Loss: 0.9494
Epoch [81/100], Loss: 0.9250
Epoch [91/100], Loss: 0.8982
TF-IDF Weighted Word2Vec + MLP Performance:
              precision    recall  f1-score   support

           0       0.64      0.77      0.70       266
           1       0.67      0.78      0.72       276
           2       0.67      0.60      0.63       285
           3       0.69      0.42      0.52       171

    accuracy                           0.66       998
   macro avg       0.67      0.64      0.64       998
weighted avg       0.67      0.66      0.66       998



0.6643286573146293

tensor([[0.2454, 0.6158, 0.0849, 0.0254, 0.0285],
        [0.3026, 0.4419, 0.0476, 0.1130, 0.0949],
        [0.2641, 0.1971, 0.4325, 0.0186, 0.0877]])